In [1]:
# import libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

In [3]:
# import data

df = pd.read_csv("/content/drive/MyDrive/DS Course Uploads/Datasets/waterQuality1.csv")
df.head()

,aluminium,ammonia,arsenic,barium,cadmium,chloramine,chromium,copper,flouride,bacteria,...,lead,nitrates,nitrites,mercury,perchlorate,radium,selenium,silver,uranium,is_safe
0,1.65,9.08,0.04,2.85,0.007,0.35,0.83,0.17,0.05,0.20,...,0.054,16.08,1.13,0.007,37.75,6.78,0.08,0.34,0.02,1
1,2.32,21.16,0.01,3.31,0.002,5.28,0.68,0.66,0.90,0.65,...,0.100,2.01,1.93,0.003,32.26,3.21,0.08,0.27,0.05,1
2,1.01,14.02,0.04,0.58,0.008,4.24,0.53,0.02,0.99,0.05,...,0.078,14.16,1.11,0.006,50.28,7.07,0.07,0.44,0.01,0
3,1.36,11.33,0.04,2.96,0.001,7.23,0.03,1.66,1.08,0.71,...,0.016,1.41,1.29,0.004,9.12,1.72,0.02,0.45,0.05,1
4,0.92,24.33,0.03,0.20,0.006,2.67,0.69,0.57,0.61,0.13,...,0.117,6.74,1.11,0.003,16.90,2.41,0.02,0.06,0.02,1


### EDA

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7999 entries, 0 to 7998
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   aluminium    7999 non-null   float64
 1   ammonia      7999 non-null   object 
 2   arsenic      7999 non-null   float64
 3   barium       7999 non-null   float64
 4   cadmium      7999 non-null   float64
 5   chloramine   7999 non-null   float64
 6   chromium     7999 non-null   float64
 7   copper       7999 non-null   float64
 8   flouride     7999 non-null   float64
 9   bacteria     7999 non-null   float64
 10  viruses      7999 non-null   float64
 11  lead         7999 non-null   float64
 12  nitrates     7999 non-null   float64
 13  nitrites     7999 non-null   float64
 14  mercury      7999 non-null   float64
 15  perchlorate  7999 non-null   float64
 16  radium       7999 non-null   float64
 17  selenium     7999 non-null   float64
 18  silver       7999 non-null   float64
 19  uraniu

In [5]:
df['ammonia'] = pd.to_numeric(df['ammonia'], errors='coerce')
df.ammonia = df.ammonia.astype(float)

In [6]:
df.duplicated().sum()

0

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7999 entries, 0 to 7998
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   aluminium    7999 non-null   float64
 1   ammonia      7996 non-null   float64
 2   arsenic      7999 non-null   float64
 3   barium       7999 non-null   float64
 4   cadmium      7999 non-null   float64
 5   chloramine   7999 non-null   float64
 6   chromium     7999 non-null   float64
 7   copper       7999 non-null   float64
 8   flouride     7999 non-null   float64
 9   bacteria     7999 non-null   float64
 10  viruses      7999 non-null   float64
 11  lead         7999 non-null   float64
 12  nitrates     7999 non-null   float64
 13  nitrites     7999 non-null   float64
 14  mercury      7999 non-null   float64
 15  perchlorate  7999 non-null   float64
 16  radium       7999 non-null   float64
 17  selenium     7999 non-null   float64
 18  silver       7999 non-null   float64
 19  uraniu

In [8]:
df.is_safe.value_counts()

,count
is_safe,
0,7084
1,912
#NUM!,3


In [9]:
df.drop(df[df.is_safe == "#NUM!"].index, inplace=True)

In [10]:
df.is_safe.value_counts()

,count
is_safe,
0,7084
1,912


In [11]:
df['is_safe'] = pd.to_numeric(df['is_safe'], errors='coerce')
df['is_safe'] = df['is_safe'].astype(int)

In [12]:
# Changing values to 0 and 1 using condition given in problem statement

# mapping = {"aluminium": 2.8, "ammonia": 32.5, "arsenic": 0.01, "barium": 2, "cadmium": 0.005, "chloramine": 4, "chromium": 0.1, "copper": 1.3, "flouride": 1.5, "bacteria": 0, "viruses": 0, "lead": 0.015, "nitrates": 10, "nitrites": 1, "mercury": 0.002, "perchlorate": 56, "radium": 5, "selenium": 0.5, "silver": 0.1, "uranium": 0.3}

# for col in df.columns[:-1]:
#   df[col] = df[col].apply(lambda x: 0 if int(x) > mapping[col] else 1)

In [13]:
df.head()

,aluminium,ammonia,arsenic,barium,cadmium,chloramine,chromium,copper,flouride,bacteria,...,lead,nitrates,nitrites,mercury,perchlorate,radium,selenium,silver,uranium,is_safe
0,1.65,9.08,0.04,2.85,0.007,0.35,0.83,0.17,0.05,0.20,...,0.054,16.08,1.13,0.007,37.75,6.78,0.08,0.34,0.02,1
1,2.32,21.16,0.01,3.31,0.002,5.28,0.68,0.66,0.90,0.65,...,0.100,2.01,1.93,0.003,32.26,3.21,0.08,0.27,0.05,1
2,1.01,14.02,0.04,0.58,0.008,4.24,0.53,0.02,0.99,0.05,...,0.078,14.16,1.11,0.006,50.28,7.07,0.07,0.44,0.01,0
3,1.36,11.33,0.04,2.96,0.001,7.23,0.03,1.66,1.08,0.71,...,0.016,1.41,1.29,0.004,9.12,1.72,0.02,0.45,0.05,1
4,0.92,24.33,0.03,0.20,0.006,2.67,0.69,0.57,0.61,0.13,...,0.117,6.74,1.11,0.003,16.90,2.41,0.02,0.06,0.02,1


In [14]:
# StandardScaler the data

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df.drop('is_safe', axis=1))
df_scaled = pd.DataFrame(df_scaled, columns=df.columns[:-1])

### Modeling

In [15]:
# Separating input and output

X = df_scaled
y = df['is_safe']

In [16]:
# Perform train test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# KFold

k_fold = KFold(n_splits=10, shuffle=True, random_state=42)

In [18]:
rf = RandomForestClassifier(random_state=42)
rf_fit = rf.fit(X_train, y_train)
y_pred = rf_fit.predict(X_test)

In [19]:
# Cross Validation

cv_scores = cross_val_score(rf, X, y, cv=k_fold)
cv_scores.mean()

0.9603556007509386

In [20]:
print("Accuracy - ", metrics.accuracy_score(y_pred, y_test))
print("Precision - ", metrics.precision_score(y_pred, y_test))
print("Recall - ", metrics.recall_score(y_pred, y_test))
print("F1 Score - ", metrics.f1_score(y_pred, y_test))

Accuracy -  0.960625
Precision -  0.705
Recall -  0.9724137931034482
F1 Score -  0.817391304347826


In [21]:
# Training score evaluation

y_train_pred = rf_fit.predict(X_train)

print("Accuracy - ", metrics.accuracy_score(y_train_pred, y_train))
print("Precision - ", metrics.precision_score(y_train_pred, y_train))
print("Recall - ", metrics.recall_score(y_train_pred, y_train))
print("F1 Score - ", metrics.f1_score(y_train_pred, y_train))

Accuracy -  1.0
Precision -  1.0
Recall -  1.0
F1 Score -  1.0


In [22]:
# Apply PCA to get 5 principal components

pca = PCA(n_components=5)
X_pca = pca.fit_transform(X)

# Step 4: Convert the PCA-transformed data back into a DataFrame
df_pca = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(5)])
df_pca.head()

,PC1,PC2,PC3,PC4,PC5
0,2.167489,-1.388131,-0.890922,1.391953,-0.555493
1,2.970829,1.258655,-0.965331,0.769768,-0.096268
2,2.083287,-1.415599,-1.076970,1.446839,-0.393981
3,1.705591,1.810150,-0.887939,0.686323,1.964988
4,-0.137867,-0.665006,-1.257430,1.088963,0.471565


In [26]:
importances = rf.feature_importances_
feature_importance_df = pd.DataFrame({'feature': X.columns, 'importance': importances})
feature_importance_df.sort_values(by='importance', ascending=False)

,feature,importance
0,aluminium,0.203056
4,cadmium,0.116255
15,perchlorate,0.112092
2,arsenic,0.063197
1,ammonia,0.049731
5,chloramine,0.048406
18,silver,0.047536
12,nitrates,0.040542
13,nitrites,0.034229
16,radium,0.033760


GridSearchCV is quite time consuming even with PCA. So commented it.

Accuracy increased by 0.7 only but iverfitting reduced.

In [52]:
# # Use GridSearchCV algorithm to improve accuracy (trying doing PCA and put that in here)

# # Define parameters for RandomForestClassifier
# param_grid = {
#     'n_estimators': [50, 100],           # Number of trees in the forest
#     'max_depth': [10, 20],          # Maximum depth of the tree
#     'min_samples_split': [2, 5],           # Minimum number of samples required to split an internal node
#     'min_samples_leaf': [1, 2],             # Minimum number of samples required to be at a leaf node
#     'bootstrap': [True, False]                 # Whether bootstrap samples are used when building trees
# }

# # Initialize the Grid Search with Cross-Validation
# grid_search = GridSearchCV(rf, param_grid, cv=k_fold)
# grid_search.fit(X_train, y_train)
# # grid_search.fit(df_pca, y)

# # Print the best parameters and best score
# print(f"Best Parameters: {grid_search.best_params_}")
# print(f"Best Cross-Validation Score: {grid_search.best_score_:.4f}")

Best Parameters: {'bootstrap': False, 'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
Best Cross-Validation Score: 0.9675


In [23]:
# Optimized model

rf_opt = RandomForestClassifier(random_state=42, bootstrap=False, max_depth=20, min_samples_leaf=2, min_samples_split=5, n_estimators=100)
rf_opt_fit = rf_opt.fit(X_train, y_train)
y_opt_pred = rf_opt_fit.predict(X_test)

# Cross Validation

cv_scores = cross_val_score(rf_opt, X, y, cv=k_fold)
cv_scores.mean()

0.9674831038798498

In [24]:
# Evaluation

print("Accuracy - ", metrics.accuracy_score(y_opt_pred, y_test))
print("Precision - ", metrics.precision_score(y_opt_pred, y_test))
print("Recall - ", metrics.recall_score(y_opt_pred, y_test))
print("F1 Score - ", metrics.f1_score(y_opt_pred, y_test))

# Training score evaluation

y_train_opt_pred = rf_opt_fit.predict(X_train)

print("Accuracy - ", metrics.accuracy_score(y_train_opt_pred, y_train))
print("Precision - ", metrics.precision_score(y_train_opt_pred, y_train))
print("Recall - ", metrics.recall_score(y_train_opt_pred, y_train))
print("F1 Score - ", metrics.f1_score(y_train_opt_pred, y_train))

Accuracy -  0.964375
Precision -  0.765
Recall -  0.9386503067484663
F1 Score -  0.8429752066115703
Accuracy -  0.9996873045653534
Precision -  0.9971910112359551
Recall -  1.0
F1 Score -  0.9985935302390998


In [25]:
importances_opt = rf_opt.feature_importances_
feature_importance_opt_df = pd.DataFrame({'feature': X.columns, 'importance': importances_opt})
feature_importance_opt_df.sort_values(by='importance', ascending=False)

,feature,importance
0,aluminium,0.213424
15,perchlorate,0.129967
4,cadmium,0.122605
2,arsenic,0.064170
18,silver,0.052417
1,ammonia,0.047677
5,chloramine,0.042909
19,uranium,0.038306
12,nitrates,0.035657
16,radium,0.032795
